In [2]:
import cv2
import numpy as np

In [1]:
import torch

In [3]:
def try_func(func, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        print(e)
        return None

In [4]:
import cmapy
cmap = cmapy.cmap('nipy_spectral')

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../models/handmotion/epoch5.pt')
model.to(device)

Using cache found in C:\Users\xkfkf/.cache\torch\hub\ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...

requirements: 1 package updated per C:\Users\xkfkf\.cache\torch\hub\ultralytics_yolov5_master\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  2023-3-27 Python-3.11.1 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)



Exception: [Errno 2] No such file or directory: '..\\models\\handmotion\\epoch5.pt'. Cache may be out of date, try `force_reload=True` or see https://github.com/ultralytics/yolov5/issues/36 for help.

In [7]:
colors = [
    (0, 0, 255),
    (0, 255, 0),
    (255, 0, 0),
]

In [8]:
cap = cv2.VideoCapture(0)
def test():
    fps = cap.get(cv2.CAP_PROP_FPS) # 프레임 수 구하기
    delay = int(1000/fps)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    history = np.zeros((height, width), dtype=np.int16)

    # 배경 제거 객체
    fgbg = cv2.createBackgroundSubtractorMOG2()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == False:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # 배경 제거 마스크 계산
        fgmask = fgbg.apply(frame)

        # make 24 frame of history based on fgmask
        pos = 15
        history = np.where(fgmask == 255, 255 + pos, history)
        history = np.where(history > -100, history - pos, history)

        # make clip for safety
        history = np.clip(history, 0, 255)
        history_frame = history.astype(np.uint8)

        # color map
        color_map_history = cv2.applyColorMap(history_frame, cmap)

        # detect
        results = model(color_map_history, size=640)

        result = frame.copy()
        for i in range(len(results.xyxy[0])):
            r = results.pandas().xyxy[0].iloc[i, :].values.tolist()
            
            x1, y1, x2, y2, confidence, cls_id, cls_name = r
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            c = colors[cls_id]
            name = f"{cls_id}. {cls_name} : {confidence:.2f}"

            cv2.rectangle(result, (x1, y1), (x2, y2), c, 2)
            cv2.putText(result, name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 2)


        cv2.imshow('frame',frame)
        cv2.imshow('bgsub',fgmask)
        cv2.imshow('history', history_frame)
        cv2.imshow('cmaped', color_map_history)
        cv2.imshow('mhi_box', np.squeeze(results.render()))
        cv2.imshow('result', result)

        if cv2.waitKey(1) & 0xff == 27:
            break
try:
    test()
except Exception as e:
    raise e
finally:
    try_func(cap.release)
    try_func(cv2.destroyAllWindows)


print("Goodbye User")

NameError: name 'model' is not defined

In [9]:
try_func(cap.release)
try_func(cv2.destroyAllWindows)